In [1]:
authorList = [551, 2703, 2971, 8303, 7679] 
doc_id = 1210

In [2]:
parameters = {
    'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'samples': [50, 100, 150, 200, 250, 300],
    # 'dimensions': [50, 100, 200], # 300],
    'iterations': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
}
results = {
    'dropout': [0, 0, 0, 0, 0, 0],
    'samples': [0, 0, 0, 0, 0, 0],
    # 'dimensions': [0, 0, 0, 0],
    'iterations': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}
level = 'char'
for idxp, paralist in parameters.iteritems():
    if idxp == 'dropout':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = 300, 
                                                                         dimensions = 100,
                                                                         dropout = val
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[0])))
            results[idxp].insert(idxl, predY[0])
        
    
    if idxp == 'samples':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = val, 
                                                                         dimensions = 100,
                                                                         dropout = 0.4
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[0])))
            results[idxp].insert(idxl, predY[0])
        
    
    if idxp == 'dimensions':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = 300, 
                                                                         dimensions = val,
                                                                         dropout = 0.4
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[0])))
            results[idxp].insert(idxl, predY[0])
        
    
    if idxp == 'iterations':
        for idxl, val in enumerate(paralist):
            print("Current test: %s" % (str(idxp)))
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList, 
                                                                         doc_id = doc_id, 
                                                                         level = level, 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = val,
                                                                         samples = 300, 
                                                                         dimensions = 100,
                                                                         dropout = 0.4
                                                                        )
            print("%12s test:   index: %2s   pred: %5s" % (str(idxp), str(idxl), str(predY[0])))
            results[idxp].insert(idxl, predY[0])

Current test: dropout


Using Theano backend.
Using gpu device 0: GeForce GTX 950 (CNMeM is disabled, cuDNN 5005)


Level = Char
Indexing char vectors.
Found 69 char vectors.
Processing text dataset
chunk_size 1449.
Execution completed
Read completed
Number of rows: 113
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (30266, 2)
Mean: 6053
Min: 1045
Max: 18665
Authors [551, 2703, 2971, 7679, 8303].
Found 1500 texts.
Found 1500 labels.
Shape of data tensor: (1500, 1449, 69)
Shape of label tensor: (1500, 5)
Done compiling.
Train on 1200 samples, validate on 300 samples
Epoch 1/40
1200/1200 [==============================] - 10s - loss: 1.6094 - acc: 0.1942 - val_loss: 1.6102 - val_acc: 0.2200

2017-01-26 20:01:53,062| ERROR   | Unknown exception: [Errno 9] Bad file descriptor
2017-01-26 20:01:53,129| ERROR   | Traceback (most recent call last):
2017-01-26 20:01:53,130| ERROR   |   File "/home/ninadt/anaconda2/lib/python2.7/site-packages/paramiko/transport.py", line 1783, in run
2017-01-26 20:01:53,131| ERROR   |     self._channel_handler_table[ptype](chan, m)
2017-01-26 20:01:53,131| ERROR   |   File "/home/ninadt/anaconda2/lib/python2.7/site-packages/paramiko/channel.py", line 940, in _feed
2017-01-26 20:01:53,132| ERROR   |     self.in_buffer.feed(s)
2017-01-26 20:01:53,132| ERROR   |   File "/home/ninadt/anaconda2/lib/python2.7/site-packages/paramiko/buffered_pipe.py", line 98, in feed
2017-01-26 20:01:53,132| ERROR   |     self._event.set()
2017-01-26 20:01:53,133| ERROR   |   File "/home/ninadt/anaconda2/lib/python2.7/site-packages/paramiko/pipe.py", line 130, in set
2017-01-26 20:01:53,133| ERROR   |     self._pipe.set()
2017-01-26 20:01:53,134| ERROR   |   File "/home

author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (30266, 2)
Mean: 6053
Min: 1045
Max: 18665
Authors [551, 2703, 2971, 7679, 8303].
Found 1250 texts.
Found 1250 labels.
Shape of data tensor: (1250, 1449, 69)
Shape of label tensor: (1250, 5)
Done compiling.
Train on 1000 samples, validate on 250 samples
Epoch 1/40
1000/1000 [==============================] - 8s - loss: 1.6091 - acc: 0.2020 - val_loss: 1.6097 - val_acc: 0.1840

In [3]:
print results

{'dropout': [0.19562849, 0.2067472, 0.20520647, 0.1999238, 0.20436038, 0.19710673, 0, 0, 0, 0, 0, 0], 'iterations': [0.19566603, 0.21187821, 0.20399822, 0.19273828, 0.19091108, 0.19527607, 0.20800576, 0.19891253, 0.21076769, 0.1885933, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'samples': [0.20390888, 0.19373277, 0.19998761, 0.19763894, 0.1969312, 0.20093855, 0, 0, 0, 0, 0, 0]}


In [4]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [5]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()